# Burn a `mbtiles` file

#### Setup

_Assumes the following:_

- You have the AWS cli installed and configured with a firecares connection as the "default" profile that has:
    - The ability to invalidate CloudFront distributions
    - The ability to upload data to s3://firecares-data-backup
- You have GDAL/ogr2ogr installed
- You have tippecanoe installed
- You have the tessera-ansible project checked-out with the necessary ansible vault keys
- You have bash >= 4.0 installed
- You have plenty of space in your /tmp folder

**These cells will do the following:**

- Project the input file to EPSG:4326
- Upload the results to S3 under the county/state/location as provided
- Pull the remote geojson files from S3 that are used to create the .mbtiles output
- Generate mbtiles using tippecanoe from the collected geojson files
- Upload the output mbtiles to S3
- Re-download mbtiles and restart tileservers
- Invalidate the CloudFront tiles CDN => (a|b|c|d).firecares.org

In [ ]:
# Location of the file to process
fname = './work/Southington_fire_districts.shp'

# One of `fire_hydrants`, `fire_stations`, `fire_districts`, `building_footprints`
feature_type = 'fire_districts'

# Meta about incoming file
info = dict(country='us', state='ct', location='southington', feature_type=feature_type)

if '-' in info['location']:
    raise Exception('Location MUST use underscores vs dashes')

# Location of your tessera-ansible project
playbook_dir = '~/projects/firecares/tessera-ansible'

---

In [ ]:
import os
file_root = os.path.splitext(fname)[0]
mbtiles_mapping = {
    'fire_districts': 'districts.mbtiles',
    'fire_hydrants': 'hydrants.mbtiles',
    'fire_stations': 'stations.mbtiles'
}
mbtiles_name = mbtiles_mapping[feature_type]
layer_name = mbtiles_name.split('.')[0]
dest = '{country}/{state}/{location}/{country}-{state}-{location}-{feature_type}.geojson'.format(**info)
outf = file_root + '.geojson'
s3dest = 's3://firecares-data-backup/' + dest

print s3dest

#### Copy projected geojson to S3

In [ ]:
!rm $outf
!ogr2ogr -f "Geojson" $outf $fname -t_srs EPSG:4326
!aws s3 cp $outf $s3dest

#### Copy feature geojson from S3 to local and flatten

In [ ]:
%%bash -s "$feature_type"
rm -Rf /tmp/tiles
mkdir -p /tmp/tiles
cd /tmp/tiles
aws s3 cp s3://firecares-data-backup/ . --exclude="*" --include "*$1*.geojson" --exclude="*network_analysis*" --recursive
# You need bash >= 4.0 for globstar support => `brew install bash && echo '/usr/local/bin/bash' >> /etc/shells && chsh -s /usr/local/bin/bash`
shopt -s globstar
cd /tmp/tiles/us
cp */**/*.geojson ../

In [ ]:
%%bash -s "$mbtiles_name" "$layer_name"
cd /tmp/tiles
echo "Writing layer: $2"
echo tippecanoe -r 0 -z18 -Z14 --no-line-simplification --no-feature-limit --no-tile-size-limit --no-polygon-splitting -f -l $2 -o $1 *.geojson
tippecanoe -r 0 -z18 -Z14 --no-line-simplification --no-feature-limit --no-tile-size-limit --no-polygon-splitting --include name -f -l $2 -o $1 *.geojson

#### Upload mbiles to s3

In [ ]:
%%bash -s "$mbtiles_name"
echo Copying $1 to S3
cd /tmp/tiles
aws s3 cp s3://firecares-tiles/$1 s3://firecares-tiles/backups/$(date +"%Y%m%d-%H%M%S")-$1
aws s3 cp $1 s3://firecares-tiles/ --acl=public-read

#### Re-download mbtiles on tile servers and restart tileserver process

In [ ]:
!cd $playbook_dir && ansible-playbook -i hosts firecares-tileservers-production.yml --tags "mbtiles.reload" -e "reload_only=true" --private-key="~/.ssh/firecares-tileserver.pem"  

#### Invalidate CDN

In [ ]:
!aws cloudfront create-invalidation --distribution-id E3LIEGWQGOWXQG --paths '/*'

#### Preview

_Tested on node 4.2.2, see https://github.com/mojodna/tessera_

`tessera mbtiles://./hydrants.mbtiles`

### Burning current FireCARES station list

In [ ]:
import psycopg2
conn = psycopg2.connect('service=firecares')

sql = """
SELECT json_build_object(
    'type', 'FeatureCollection',
    'crs',  json_build_object(
        'type',      'name', 
        'properties', json_build_object(
            'name', 'urn:ogc:def:crs:OGC:1.3:CRS84')),
    'features', json_agg(
        json_build_object(
            'type',       'Feature',
            'id',         id,
            'geometry',   ST_AsGeoJSON(str.geom)::json,
            'properties', json_build_object(
                'name', str.name
            )
        )
    )
)
from firestation_firestation fs
inner join firestation_usgsstructuredata str
    on fs.usgsstructuredata_ptr_id = str.id
where fs.archived = false
    and fs.department_id is not null
"""

with conn.cursor() as c:
    c.execute(sql)
    res = c.fetchall()[0][0]

In [ ]:
import json
print len(res)
json.dump(res, open('/tmp/stations.geojson', 'w'), indent=4)

In [ ]:
!tippecanoe -r 0 -z18 -Z11 --no-line-simplification --no-feature-limit --no-tile-size-limit --no-polygon-splitting -f -l stations -o /tmp/stations.mbtiles /tmp/stations.geojson

In [ ]:
!aws s3 cp /tmp/stations.mbtiles s3://firecares-tiles/ --acl=public-read
!cd $playbook_dir && ansible-playbook -i hosts firecares-tileservers-production.yml --tags "mbtiles.reload" -e "reload_only=true" --private-key="~/.ssh/firecares-tileserver.pem"  
!aws cloudfront create-invalidation --distribution-id E3LIEGWQGOWXQG --paths '/*'